In [1]:
# 매장 정보 불러오기 위한 모듈 Import
import numpy as np
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
from urllib.request import Request

from tqdm import tqdm_notebook

import time

In [ ]:
# 웹 드라이버 구동
driver = webdriver.Chrome('chromedriver.exe')
driver.get('https://www.starbucks.co.kr/store/store_map.do?disp=locale')

In [3]:
# 서울 클릭
def SeoulClick():
    xpath_seoul = '''//*[@id="container"]/div/form/fieldset/div/section/article[1]/article/article[2]/div[1]/div[2]/ul/li[1]/a'''
    driver.find_element(By.XPATH, xpath_seoul).send_keys(Keys.ENTER)

# 지역검색 클랙
def LocaClick():
    xpath_loca = '//*[@id="container"]/div/form/fieldset/div/section/article[1]/article/header[2]/h3/a'
    driver.find_element(By.XPATH, xpath_loca).send_keys(Keys.ENTER)

SeoulClick()

In [ ]:
# 결과물 DataFrame 틀 만들기
def GenerateDataFrame(Area = [], Branch = [], Addr = []):
    tmp = (
        {'Area': Area,
        'Branch' : Branch,
        'Address' : Addr,
        }
    )
    return pd.DataFrame(tmp)

starbucks = GenerateDataFrame([], [], [])
starbucks.head()

In [5]:
# 구 내부에서 정보 가져오는 함수

def getInfo_gu(Area):
    branch = []; addr = []
    info = driver.find_elements(By.CLASS_NAME, 'quickResultLstCon')
    for i in info:
        ActionChains(driver).move_to_element(i).perform()

        li = i.text.split('\n')[:2]
        branch.append(li[0])
        addr.append(li[1])
        
    return [branch, addr]

In [ ]:
# Initial empty DataFrame: "starbucks"
starbucks = GenerateDataFrame([], [], [])

# Approaching with area number: 25 area starting from [2] '강남구' to [26] '중랑구'
for n in tqdm_notebook(range(2,27)):
    gu = driver.find_element(By.XPATH, f'//*[@id="mCSB_2_container"]/ul/li[{n}]/a')
    if n >= 23:
        ActionChains(driver).move_to_element(gu).perform()
    gu_name = gu.text # 클릭 전 구 이름 변수 지정
    gu.click() # 구 클릭
    time.sleep(1)
    
    info = getInfo_gu(gu_name)
    
    starbucks_tmp = GenerateDataFrame([gu_name for i in range(len(info[0]))], info[0], info[1])
    starbucks = pd.concat([starbucks, starbucks_tmp])
    
    # Click the button "[지역검색]"
    LocaClick()
    time.sleep(1)

    # Click the button "[서울]" 
    SeoulClick()
    time.sleep(1)

# 데이터 확인
starbucks.head()

In [ ]:
# Checking the DataFrame

starbucks.info()

총 593개의 자료가 입력됨을 확인. <br>
Concat을 활용했기 때문에 index 번호가 정리되지 않았다. <br>
reset_index를 활용해 index 정리

___

In [ ]:
# Resetting the index
starbucks.reset_index(inplace=True)

# Remove previous index
del starbucks['index']

starbucks.info()

index가 0에서 592로 잘 지정됨 확인

___


In [ ]:
# 결측치 확인
print(len(starbucks['Area'].isna())) # 구 정보 결측치 개수
print(len(starbucks['Branch'].isna())) # 지점 정보 결측치 개수
print(len(starbucks['Address'].isna())) # 주소 결측치 개수

starbucks.info()에서도 확인할 수 있었지만 (non-null), <br>
.isna() 함수를 활용해 한 번 더 추가로 확인. <br>
각 열에 결측치가 아닌 개수가 593개로, 결측치 없이 데이터 입력 성공

___

In [ ]:
# 구 정보 확인
gu_result = starbucks['Area'].unique()
print(gu_result, f'\nLength: {len(gu_result)}')

강남구부터 중량구까지 총 25개의 구가 입력된 것 확인

___

In [68]:
starbucks.to_csv('data/01_starbucks.csv', sep= ',', encoding = 'cp949')

In [69]:
driver.close()